In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from glob import glob
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import json
import random
import pandas as pd
from tqdm import tqdm
import trimesh

import torch

from ANALYSIS.analysis_utils import (
    plot_panel_info,
    visualize_meshes_plotly,
    filter_segmentation_map,
    filter_segmentation_map_clusters,
    is_clockwise,
)

from env_constants import SEWFORMER_PROJ_ROOT, DATASET_ROOT, PYGARMENT_ROOT, SEWING_DATA_ROOT

sys.path.append(PYGARMENT_ROOT)

import pygarment as pyg

In [2]:
def filter_fn(path):
    if not os.path.isdir(path):
        print(f"{path} is not a directory")
        return False
    if not os.path.exists(os.path.join(path, "static", "spec_config.json")):
        print(f"No spec_config.json in {path}")
        return False
    return True

raw_combination_path_list = sorted(glob(os.path.join(DATASET_ROOT, "sewfactory", "*")))
combination_path_list = []
for garment_path in tqdm(raw_combination_path_list):
    if filter_fn(garment_path):
        combination_path_list.append(garment_path)
len(combination_path_list)

  0%|          | 0/13707 [00:00<?, ?it/s]

100%|██████████| 13707/13707 [00:00<00:00, 131024.50it/s]

No spec_config.json in /media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/tee_sleeveless_C2EFEDVIB0_wb_pants_straight_U3CBKG8WW1


13706

In [3]:
import json
import numpy as np
import smplx
import torch
import trimesh

SMPLH_PATH = os.path.join(
    SEWFORMER_PROJ_ROOT, "Sewformer", "assets",
)

# Create SMPL-H model
model = smplx.create(
    model_path=SMPLH_PATH,
    model_type='smplh',  # Specifically use SMPL-H
    ext='pkl',
    gender='female',
    use_pca=False,  # Important: disable PCA for hand poses
    batch_size=1,
)

In [5]:
# garment_name_dict = {} # check if some garment is duplicated
# garment_name_list = []
# matching_combination_name_list = []
# panel_count_list = []
# stitch_count_list = []
# panel_name_contains_digit_list = []
# contains_clockwise_panel_list = []
# edge_type_list_list = []
# mesh_len_segmentation_match_list = []
# mesh_filter_failed_list = []
# none_vertex_count_list = []
# none_filtered_vertex_count_list = []
# stitch_vertex_count_list = []
# stitch_filtered_vertex_count_list = []

# for combination_path in tqdm(combination_path_list) :
#     spec_config_path = os.path.join(combination_path, "static", "spec_config.json")
#     with open(spec_config_path, "r") as f:
#         spec_config = json.load(open(spec_config_path, "r"))

#     combination_garment_name_list = list(map(
#         lambda x : os.path.basename(x["spec"].replace("\\", "/")),
#         spec_config.values()
#     ))

#     # mesh_list = []
#     for garment_name in combination_garment_name_list :
#         if garment_name not in garment_name_dict :
#             garment_name_dict[garment_name] = 0
#         else :
#             print(f"duplicating garment name {garment_name}")
#             break
#         garment_name_dict[garment_name] += 1
        
#         garment_name_list.append(garment_name)

#         matching_combination_name_list.append(os.path.basename(combination_path))

#         spec_file_path = os.path.join(
#             combination_path, "static", f"{garment_name}_specification.json"
#         )
#         pattern = pyg.pattern.wrappers.VisPattern(spec_file_path)
#         drawn_pattern_list = list(map(
#             lambda pannel_name : pattern._draw_a_panel(
#                 pannel_name, apply_transform=False, fill=True
#             ),
#             pattern.panel_order()
#         ))
#         panel_svg_path_dict = {
#             panel_name : pattern._draw_a_panel(
#                 panel_name, apply_transform=False, fill=True
#             )
#             for panel_name in pattern.panel_order()
#         }
#         panel_count_list.append(len(panel_svg_path_dict))

#         stitch_dict = {
#             i : v for i, v in enumerate(pattern.pattern['stitches'])
#         }
#         stitch_count_list.append(len(stitch_dict))

#         # find if panel name contains digit
#         panel_name_contains_digit = any(map(
#             lambda x : any(char.isdigit() for char in x),
#             pattern.panel_order()
#         ))
#         panel_name_contains_digit_list.append(panel_name_contains_digit)


#         contains_clockwise_panel = False
#         for panel_name, panel in panel_svg_path_dict.items() :
#             point_list = []
#             edges = list(panel[0])
#             for edge in edges :
#                 point_list.append((edge.start.real, edge.start.imag))
#             if is_clockwise(point_list) :
#                 contains_clockwise_panel = True
#                 break
#         contains_clockwise_panel_list.append(contains_clockwise_panel)
            
            
#         edge_type_set = set()
#         for panel_name, panel in panel_svg_path_dict.items() :
#             for edge in panel :
#                 if type(edge) not in edge_type_set :
#                     edge_type_set.add(type(edge))
#         edge_type_list_list.append(list(edge_type_set))

#         mesh = trimesh.load_mesh(os.path.join(
#             combination_path, "static", f"{garment_name}_{garment_name}.obj"
#         ))

#         with open(
#             os.path.join(
#                 combination_path, "static", f"{garment_name}_{garment_name}_segmentation.txt"
#             ),
#             "r"
#         ) as f:
#             mesh_segmentation_list = list(map(
#                 lambda x : x.strip(),
#                 f.readlines()
#             ))
        
#         if len(mesh_segmentation_list) != mesh.vertices.shape[0] :    
#             print(f"{combination_path} {garment_name} {mesh.vertices.shape[0]} {len(mesh_segmentation_list)}")
#             mesh_len_segmentation_match_list.append(False)
#         else :
#             mesh_len_segmentation_match_list.append(True)
            
#         none_vertex_count_list.append(
#             (np.array(mesh_segmentation_list) == "None").sum()
#         )
#         stitch_vertex_count_list.append(
#             (np.array(mesh_segmentation_list) == "stitch").sum()
#         )
        
        
#         filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, mesh_segmentation_list)
#         filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
#         filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
#         filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
#         filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
        
#         vertex_mask_dict = {}
#         for panel_name in panel_svg_path_dict.keys() :
#             vertex_mask_dict[panel_name] = np.array(list(map(
#                 lambda x : x == panel_name,
#                 mesh_segmentation_list
#             )))
#         vertex_mask_dict["stitch"] = np.array(list(map(
#             lambda x : x == "stitch",
#             mesh_segmentation_list
#         )))
#         vertex_mask_dict["None"] = np.array(list(map(
#             lambda x : x == "None",
#             mesh_segmentation_list
#         )))
        

#         filtered_vertex_mask_dict = {}
#         for panel_name in panel_svg_path_dict.keys() :
#             filtered_vertex_mask_dict[panel_name] = np.array(list(map(
#                 lambda x : x == panel_name,
#                 filtered_mesh_segmentation_list
#             )))
        
#         mesh_filter_failed = False
        
#         stitch_mask = np.array(list(map(
#             lambda x : x == "stitch",
#             filtered_mesh_segmentation_list
#         )))
#         stitch_filtered_vertex_count_list.append(
#             stitch_mask.sum()
#         )
#         if stitch_mask.sum() > 0 :
#             mesh_filter_failed = True
#             filtered_vertex_mask_dict["stitch"] = stitch_mask
        
#         none_mask = np.array(list(map(
#             lambda x : x == "None",
#             filtered_mesh_segmentation_list
#         )))
#         none_filtered_vertex_count_list.append(
#             none_mask.sum()
#         )
#         if none_mask.sum() > 0 :
#             mesh_filter_failed = True
#             filtered_vertex_mask_dict["None"] = none_mask
        
#         mesh_filter_failed_list.append(mesh_filter_failed)
        
#         with open(
#             os.path.join(
#                 combination_path, "static", f"{garment_name}_{garment_name}_segmentation_filtered.txt"
#             ),
#             "w"
#         ) as f :
#             f.write("\n".join(filtered_mesh_segmentation_list))
#             f.write("\n")
        
    
# garment_df = pd.DataFrame(
#     {
#         "garment_name" : garment_name_list,
#         "matching_combination_name" : matching_combination_name_list,
#         "panel_count" : panel_count_list,
#         "stitch_count" : stitch_count_list,
#         "panel_name_contains_digit" : panel_name_contains_digit_list,
#         "contains_clockwise_panel" : contains_clockwise_panel_list,
#         # "edge_type_list" : edge_type_list_list,
#         "mesh_len_segmentation_match" : mesh_len_segmentation_match_list,   
#         "mesh_filter_failed" : mesh_filter_failed_list,
#         "none_vertex_count" : none_vertex_count_list,
#         "stitch_vertex_count" : stitch_vertex_count_list,
#         "none_filtered_vertex_count" : none_filtered_vertex_count_list,
#         "stitch_filtered_vertex_count" : stitch_filtered_vertex_count_list,
#     }
# )
# garment_df.to_csv(
#     f"garment_df_{time.strftime('%Y%m%d_%H%M%S')}.csv",
#     index=False
# )
# garment_df

In [6]:
garment_name_dict = {} # check if some garment is duplicated
garment_name_list = []
matching_combination_name_list = []
panel_count_list = []
stitch_count_list = []
panel_name_contains_digit_list = []
contains_clockwise_panel_list = []
edge_type_list_list = []
mesh_len_segmentation_match_list = []
mesh_filter_failed_list = []
none_vertex_count_list = []
none_filtered_vertex_count_list = []
stitch_vertex_count_list = []
stitch_filtered_vertex_count_list = []

for combination_path in tqdm(combination_path_list) :
    spec_config_path = os.path.join(combination_path, "static", "spec_config.json")
    with open(spec_config_path, "r") as f:
        spec_config = json.load(open(spec_config_path, "r"))

    combination_garment_name_list = list(map(
        lambda x : os.path.basename(x["spec"].replace("\\", "/")),
        spec_config.values()
    ))

    try :
        # mesh_list = []
        for garment_name in combination_garment_name_list :
            if garment_name not in garment_name_dict :
                garment_name_dict[garment_name] = 0
            else :
                print(f"duplicating garment name {garment_name}")
                break
            garment_name_dict[garment_name] += 1
            


            spec_file_path = os.path.join(
                combination_path, "static", f"{garment_name}_specification.json"
            )
            pattern = pyg.pattern.wrappers.VisPattern(spec_file_path)
            drawn_pattern_list = list(map(
                lambda pannel_name : pattern._draw_a_panel(
                    pannel_name, apply_transform=False, fill=True
                ),
                pattern.panel_order()
            ))
            panel_svg_path_dict = {
                panel_name : pattern._draw_a_panel(
                    panel_name, apply_transform=False, fill=True
                )
                for panel_name in pattern.panel_order()
            }

            stitch_dict = {
                i : v for i, v in enumerate(pattern.pattern['stitches'])
            }

            # find if panel name contains digit
            panel_name_contains_digit = any(map(
                lambda x : any(char.isdigit() for char in x),
                pattern.panel_order()
            ))


            contains_clockwise_panel = False
            for panel_name, panel in panel_svg_path_dict.items() :
                point_list = []
                edges = list(panel[0])
                for edge in edges :
                    point_list.append((edge.start.real, edge.start.imag))
                if is_clockwise(point_list) :
                    contains_clockwise_panel = True
                    break
                
                
            edge_type_set = set()
            for panel_name, panel in panel_svg_path_dict.items() :
                for edge in panel :
                    if type(edge) not in edge_type_set :
                        edge_type_set.add(type(edge))

            mesh = trimesh.load_mesh(os.path.join(
                combination_path, "static", f"{garment_name}_{garment_name}.obj"
            ))

            with open(
                os.path.join(
                    combination_path, "static", f"{garment_name}_{garment_name}_segmentation.txt"
                ),
                "r"
            ) as f:
                mesh_segmentation_list = list(map(
                    lambda x : x.strip(),
                    f.readlines()
                ))
            
            
            if len(mesh_segmentation_list) != mesh.vertices.shape[0] :    
                print(f"{combination_path} {garment_name} {mesh.vertices.shape[0]} {len(mesh_segmentation_list)}")
                mesh_len_segmentation_match = False
            else :
                mesh_len_segmentation_match = True
                
            
            
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, mesh_segmentation_list)
            filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
            filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
            filtered_mesh_segmentation_list = filter_segmentation_map_clusters( mesh, filtered_mesh_segmentation_list, threshold=80)
            filtered_mesh_segmentation_list = filter_segmentation_map(          mesh, filtered_mesh_segmentation_list)
            
            vertex_mask_dict = {}
            for panel_name in panel_svg_path_dict.keys() :
                vertex_mask_dict[panel_name] = np.array(list(map(
                    lambda x : x == panel_name,
                    mesh_segmentation_list
                )))
            vertex_mask_dict["stitch"] = np.array(list(map(
                lambda x : x == "stitch",
                mesh_segmentation_list
            )))
            vertex_mask_dict["None"] = np.array(list(map(
                lambda x : x == "None",
                mesh_segmentation_list
            )))
            

            filtered_vertex_mask_dict = {}
            for panel_name in panel_svg_path_dict.keys() :
                filtered_vertex_mask_dict[panel_name] = np.array(list(map(
                    lambda x : x == panel_name,
                    filtered_mesh_segmentation_list
                )))
            
            mesh_filter_failed = False
            
            stitch_mask = np.array(list(map(
                lambda x : x == "stitch",
                filtered_mesh_segmentation_list
            )))
            if stitch_mask.sum() > 0 :
                mesh_filter_failed = True
                filtered_vertex_mask_dict["stitch"] = stitch_mask
            
            none_mask = np.array(list(map(
                lambda x : x == "None",
                filtered_mesh_segmentation_list
            )))
            if none_mask.sum() > 0 :
                mesh_filter_failed = True
                filtered_vertex_mask_dict["None"] = none_mask
            
            
            with open(
                os.path.join(
                    combination_path, "static", f"{garment_name}_{garment_name}_segmentation_filtered.txt"
                ),
                "w"
            ) as f :
                f.write("\n".join(filtered_mesh_segmentation_list))
                f.write("\n")
        
            
            garment_name_list.append(garment_name)
            matching_combination_name_list.append(os.path.basename(combination_path))
            panel_count_list.append(len(panel_svg_path_dict))
            stitch_count_list.append(len(stitch_dict))
            panel_name_contains_digit_list.append(panel_name_contains_digit)
            contains_clockwise_panel_list.append(contains_clockwise_panel)
            edge_type_list_list.append(list(edge_type_set))
            mesh_len_segmentation_match_list.append(mesh_len_segmentation_match)
            none_vertex_count_list.append(
                (np.array(mesh_segmentation_list) == "None").sum()
            )
            stitch_vertex_count_list.append(
                (np.array(mesh_segmentation_list) == "stitch").sum()
            )
            stitch_filtered_vertex_count_list.append(
                stitch_mask.sum()
            )
            none_filtered_vertex_count_list.append(
                none_mask.sum()
            )
            mesh_filter_failed_list.append(mesh_filter_failed)
    except Exception as e :
        print(f"{combination_path}")
        print(f"{garment_name}")
        print(f"{e}")
        print()

    
garment_df = pd.DataFrame(
    {
        "garment_name" : garment_name_list,
        "matching_combination_name" : matching_combination_name_list,
        "panel_count" : panel_count_list,
        "stitch_count" : stitch_count_list,
        "panel_name_contains_digit" : panel_name_contains_digit_list,
        "contains_clockwise_panel" : contains_clockwise_panel_list,
        # "edge_type_list" : edge_type_list_list,
        "mesh_len_segmentation_match" : mesh_len_segmentation_match_list,   
        "mesh_filter_failed" : mesh_filter_failed_list,
        "none_vertex_count" : none_vertex_count_list,
        "stitch_vertex_count" : stitch_vertex_count_list,
        "none_filtered_vertex_count" : none_filtered_vertex_count_list,
        "stitch_filtered_vertex_count" : stitch_filtered_vertex_count_list,
    }
)
garment_df.to_csv(
    f"garment_df_{time.strftime('%Y%m%d_%H%M%S')}.csv",
    index=False
)
garment_df

 18%|█▊        | 2512/13706 [51:36<2:51:21,  1.09it/s]

/media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/jumpsuit_sleeveless_AK33VX0UB0 jumpsuit_sleeveless_AK33VX0UB0 7596 7597
/media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/jumpsuit_sleeveless_AK33VX0UB0
jumpsuit_sleeveless_AK33VX0UB0
list index out of range



 19%|█▉        | 2591/13706 [52:58<2:21:06,  1.31it/s]

/media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/jumpsuit_sleeveless_BGEXKZVPE0 jumpsuit_sleeveless_BGEXKZVPE0 9433 9434
/media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/jumpsuit_sleeveless_BGEXKZVPE0
jumpsuit_sleeveless_BGEXKZVPE0
list index out of range



 19%|█▉        | 2629/13706 [53:34<2:30:03,  1.23it/s]

...

/media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/wb_dress_sleeveless_KGZ0LJOS8K wb_dress_sleeveless_KGZ0LJOS8K 14314 14388


 92%|█████████▏| 12559/13706 [4:47:11<23:25,  1.23s/it]

/media/hjp/05aba9a7-0e74-4e54-9bc9-5f11b9c4c757/SewFactory/sewfactory/sewfactory/wb_dress_sleeveless_KGZ0LJOS8K
wb_dress_sleeveless_KGZ0LJOS8K
list index out of range



100%|██████████| 13706/13706 [5:16:54<00:00,  1.39s/it]


,garment_name,matching_combination_name,panel_count,stitch_count,panel_name_contains_digit,contains_clockwise_panel,mesh_len_segmentation_match,mesh_filter_failed,none_vertex_count,stitch_vertex_count,none_filtered_vertex_count,stitch_filtered_vertex_count
0,dress_sleeveless_005CWF73E9,dress_sleeveless_005CWF73E9,4,10,False,False,True,False,222,218,0,0
1,dress_sleeveless_00F3PNYG6L,dress_sleeveless_00F3PNYG6L,4,10,False,False,True,False,230,226,0,0
2,dress_sleeveless_00M2CX1YWW,dress_sleeveless_00M2CX1YWW,4,10,False,False,True,False,242,238,0,0
3,dress_sleeveless_013LZJLUQ0,dress_sleeveless_013LZJLUQ0,4,10,False,False,True,False,242,238,0,0
4,dress_sleeveless_014TV5GC0X,dress_sleeveless_014TV5GC0X,4,10,False,False,True,False,238,234,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19033,wb_dress_sleeveless_ZWY1JO5LQH,wb_dress_sleeveless_ZWY1JO5LQH,6,12,False,False,True,False,390,382,0,0
19034,wb_dress_sleeveless_ZXJD88A059,wb_dress_sleeveless_ZXJD88A059,6,12,False,False,True,True,432,424,80,0
19035,wb_dress_sleeveless_ZXTV8N6GF8,wb_dress_sleeveless_ZXTV8N6GF8,6,12,False,False,True,False,400,392,0,0
19036,wb_dress_sleeveless_ZYF6BVDMMC,wb_dress_sleeveless_ZYF6BVDMMC,6,12,False,False,True,False,322,314,0,0
